In [ ]:
from ultralytics import YOLO
import os
import cv2
import yaml
import torch
from tqdm import tqdm
import shutil

# 設定基準目錄
try:
    BASE_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    BASE_DIR = os.path.abspath(os.getcwd())


# 定義資料集結構
DATASET_DIR = os.path.join(BASE_DIR, "BCD")
TRAIN_IMAGES_DIR = os.path.join(DATASET_DIR, "train/images")
VAL_IMAGES_DIR = os.path.join(DATASET_DIR, "valid/images")
TEST_IMAGES_DIR = os.path.join(DATASET_DIR, "test/images")

# 檢查資料夾是否存在
def check_directory_exists(path, description):
    if not os.path.exists(path):
        raise FileNotFoundError(f"{description} not found: {path}")

check_directory_exists(TRAIN_IMAGES_DIR, "Train images folder")
check_directory_exists(VAL_IMAGES_DIR, "Validation images folder")
check_directory_exists(TEST_IMAGES_DIR, "Test images folder")

# 生成 dataset.yaml 配置文件
DATASET_YAML = os.path.join(BASE_DIR, "dataset.yaml")
dataset_info = {
    'train': TRAIN_IMAGES_DIR,
    'val': VAL_IMAGES_DIR,
    'test': TEST_IMAGES_DIR,
    'nc': 3,
    'names': ["Platelets", "RBC", "WBC"]
}

with open(DATASET_YAML, 'w') as yaml_file:
    yaml.dump(dataset_info, yaml_file, default_flow_style=False)

print(f"Generated {DATASET_YAML} configuration file.")

# 模型參數
MODEL_NAME = "yolov8n"
EPOCHS = 10
BATCH_SIZE = 8
IMAGE_SIZE = 416
SAVE_DIR = os.path.join(BASE_DIR, "runs")
TRAIN_DIR = os.path.join(SAVE_DIR, "detect", "train")

# 創建 YOLO 模型
weights_path = os.path.join(TRAIN_DIR, "weights/best.pt")
if os.path.exists(weights_path):
    print(f"Loading existing weights from {weights_path}")
    model = YOLO(weights_path)
else:
    print(f"No pre-trained weights found. Using default {MODEL_NAME}.pt weights")
    model = YOLO(f"{MODEL_NAME}.pt")

if os.path.exists(TRAIN_DIR):
    print(f"Removing existing train directory: {TRAIN_DIR}")
    shutil.rmtree(TRAIN_DIR)
os.makedirs(TRAIN_DIR, exist_ok=True)

# GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
model.to(device)

 
print("Starting training...")
try:
    model.train(
        data=DATASET_YAML,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMAGE_SIZE,
        save_dir=os.path.dirname(TRAIN_DIR),  
        project=SAVE_DIR,   
        name="detect/train",     
        exist_ok=True       
    )
    print(f"Training completed! Weights saved in {TRAIN_DIR}")
except Exception as e:
    print(f"Training failed with error: {e}")


# 測試並保存結果
RECORDING_DIR = os.path.join(BASE_DIR, "Recording")
os.makedirs(RECORDING_DIR, exist_ok=True)

print("Generating predictions for test images...")
test_images = os.listdir(TEST_IMAGES_DIR)
test_images = [os.path.join(TEST_IMAGES_DIR, img) for img in test_images[:20]]

for img_path in tqdm(test_images, desc="Processing Images"):
    results = model.predict(source=img_path, save=False, save_txt=False, conf=0.5)
    image = cv2.imread(img_path)
    result_image = results[0].plot()
    save_path = os.path.join(RECORDING_DIR, os.path.basename(img_path))
    cv2.imwrite(save_path, result_image)
    print(f"Saved: {save_path}")

print(f"Processed images saved in: {RECORDING_DIR}")

# 匯出模型為 TorchScript 格式
print("Exporting the model to TorchScript format...")
try:
    export_results = model.export(format='torchscript', imgsz=IMAGE_SIZE)
    print("Model successfully exported to TorchScript format.")
except Exception as e:
    print(f"Model export failed with error: {e}")



Generated e:\Torch_Project\YOLO\BCD_yolo_v8\dataset.yaml configuration file.
No pre-trained weights found. Using default yolov8n.pt weights
Removing existing train directory: e:\Torch_Project\YOLO\BCD_yolo_v8\runs\detect\train
Using device: cuda
Starting training...
New https://pypi.org/project/ultralytics/8.3.46 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=e:\Torch_Project\YOLO\BCD_yolo_v8\dataset.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=e:\Torch_Project\YOLO\BCD_yolo_v8\runs, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, 

train: Scanning E:\Torch_Project\YOLO\BCD_yolo_v8\BCD\train\labels.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning E:\Torch_Project\YOLO\BCD_yolo_v8\BCD\valid\labels.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]


Plotting labels to e:\Torch_Project\YOLO\BCD_yolo_v8\runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 8 dataloader workers
Logging results to e:\Torch_Project\YOLO\BCD_yolo_v8\runs\detect\train
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

Processing Images:  90%|█████████ | 18/20 [00:01<00:00, 18.03it/s]

Saved: e:\Torch_Project\YOLO\BCD_yolo_v8\Recording\BloodImage_00241_jpg.rf.00d6593ca59122287542bf819f62fb43.jpg

image 1/1 e:\Torch_Project\YOLO\BCD_yolo_v8\BCD\test\images\BloodImage_00254_jpg.rf.3783a73c7c92da8010897e8fb9d14448.jpg: 320x416 2 Plateletss, 16 RBCs, 1 WBC, 14.0ms
Speed: 5.0ms preprocess, 14.0ms inference, 2.3ms postprocess per image at shape (1, 3, 320, 416)
Saved: e:\Torch_Project\YOLO\BCD_yolo_v8\Recording\BloodImage_00254_jpg.rf.3783a73c7c92da8010897e8fb9d14448.jpg

image 1/1 e:\Torch_Project\YOLO\BCD_yolo_v8\BCD\test\images\BloodImage_00265_jpg.rf.acdcdc5c22ee42608c69240af0c5d732.jpg: 320x416 2 Plateletss, 14 RBCs, 1 WBC, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 416)


Processing Images: 100%|██████████| 20/20 [00:01<00:00, 13.44it/s]

Saved: e:\Torch_Project\YOLO\BCD_yolo_v8\Recording\BloodImage_00265_jpg.rf.acdcdc5c22ee42608c69240af0c5d732.jpg
Processed images saved in: e:\Torch_Project\YOLO\BCD_yolo_v8\Recording
Exporting the model to TorchScript format...
Ultralytics 8.3.44  Python-3.8.18 torch-2.0.0+cu118 CPU (12th Gen Intel Core(TM) i7-12700H)



PyTorch: starting from 'e:\Torch_Project\YOLO\BCD_yolo_v8\runs\detect\train\weights\best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 7, 3549) (5.9 MB)

TorchScript: starting export with torch 2.0.0+cu118...
TorchScript: export success  1.9s, saved as 'e:\Torch_Project\YOLO\BCD_yolo_v8\runs\detect\train\weights\best.torchscript' (11.8 MB)

Export complete (2.1s)
Results saved to E:\Torch_Project\YOLO\BCD_yolo_v8\runs\detect\train\weights
Predict:         yolo predict task=detect model=e:\Torch_Project\YOLO\BCD_yolo_v8\runs\detect\train\weights\best.torchscript imgsz=416  
Validate:        yolo val task=detect model=e:\Torch_Project\YOLO\BCD_yolo_v8\runs\detect\train\weights\best.torchscript imgsz=416 data=e:\Torch_Project\YOLO\BCD_yolo_v8\dataset.yaml  
Visualize:       https://netron.app
Model successfully exported to TorchScript format.
